In [12]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2

In [ ]:
dataset_path = "./data/cat_dog/"
cat_dog_csv = pd.read_csv(os.path.join(dataset_path, "cat_dog.csv"))
# print(cat_dog_csv.head())
img_name, labels = [], []
for i in range(len(cat_dog_csv)):
    img_name.append(cat_dog_csv.iloc[i, 0])
    labels.append(cat_dog_csv.iloc[i, 1])

['dog.11289.jpg', 'cat.10836.jpg', 'cat.7970.jpg', 'cat.2212.jpg', 'dog.2653.jpg']
[1, 0, 0, 0, 1]


In [17]:
img_all = []
shapes = []
for i in range(len(img_name)):
    img_path = os.path.join(dataset_path, "cat_dog", img_name[i])
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img_all.append(img)
    shapes.append(img.shape)

[[224 224   3]]
[1.]
